In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from bagpy import bagreader
from rosbags.image import message_to_cvimage
from collections import defaultdict
import cv2 as cv
import open3d as o3d
import detectron2
import sensor_msgs.point_cloud2 as pc2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import pyrealsense2 as rs2
if (not hasattr(rs2, 'intrinsics')):
    import pyrealsense2.pyrealsense2 as rs2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [24]:
path = "pedestrian_00.bag"
bag = bagreader(path).reader
def msgs(bag):
    topic_list = [
        "/camera_floor_left/driver/color/image_raw",
        "/camera_floor_left/driver/depth/image_rect_raw",
        "/camera_floor_left/driver/infra1/image_rect_raw",
        "/camera_floor_right/driver/color/image_raw",
        "/camera_floor_left/floor",
        "/camera_floor_left/obstacles",
        "/b_scan",
        "/b_scan_footspr_filter"
    ]
    image_l_msgs = []
    depth_l_msgs = []
    infr_l_msgs = []
    image_r_msgs = []
    pc_floor_msgs = []
    pc_obs_msgs = []
    scan_msgs = []
    scan_foot_msgs = []

    counter = np.asarray([100] * 8)

    for topic, msg, t in bag.read_messages(topics=topic_list):
        
        #if np.any(counter > 0):
            if topic == "/camera_floor_left/driver/color/image_raw":
                image_l_msgs.append(msg)
                counter[0] -= 1

            if topic == "/camera_floor_left/driver/depth/image_rect_raw":
                depth_l_msgs.append(msg)
                counter[1] -= 1

            if topic =="/camera_floor_left/driver/infra1/image_rect_raw":
                infr_l_msgs.append(msg)
                counter[2] -= 1

            if topic == "/camera_floor_right/driver/color/image_raw":
                image_r_msgs.append(msg)
                counter[3] -= 1

            if topic == "/camera_floor_left/floor":
                pc_floor_msgs.append(msg)
                counter[4] -=1

            if topic == "/camera_floor_left/obstacles":
                pc_obs_msgs.append(msg)
                counter[5] -= 1

            if topic == "/b_scan":
                scan_msgs.append(msg)
                counter[6] -= 1

            if topic == "/b_scan_footspr_filter":
                scan_foot_msgs.append(msg)
                counter[7] -= 1
        #else:
         #       break


    bag.close()

    data_params = {
        "image_l_msgs": image_l_msgs,
        "depth_l_msgs": depth_l_msgs,
        "infr_l_msgs": infr_l_msgs,
        "image_r_msgs": image_r_msgs,
        "pc_floor_msgs": pc_floor_msgs,
        "pc_obs_msgs": pc_obs_msgs,
        "scan_msgs": scan_msgs,
        "scan_foot_msgs": scan_foot_msgs,
    }

    return data_params
data_params = msgs(bag)

[INFO]  Data folder pedestrian_00 already exists. Not creating.


In [58]:
img_msg = data_params["image_r_msgs"][256]
img = message_to_cvimage(img_msg)
#plt.axis('off')
#plt.imshow(img)
print(img_msg)

header: 
  seq: 2687
  stamp: 
    secs: 1720112669
    nsecs: 444463800
  frame_id: "camera_floor_right_color_optical_frame"
height: 240
width: 424
encoding: "rgb8"
is_bigendian: 0
step: 1272
data: [119, 138, 167, 119, 138, 167, 118, 140, 166, 112, 134, 160, 115, 137, 163, 119, 141, 167, 117, 135, 160, 118, 136, 161, 115, 131, 152, 122, 138, 159, 118, 143, 164, 115, 140, 161, 116, 136, 162, 119, 139, 165, 122, 137, 163, 123, 138, 164, 114, 128, 148, 117, 131, 151, 119, 134, 149, 120, 135, 150, 117, 135, 142, 113, 131, 138, 114, 138, 140, 119, 143, 145, 117, 135, 144, 113, 131, 140, 115, 128, 137, 118, 131, 140, 119, 133, 137, 109, 123, 127, 116, 124, 125, 112, 120, 121, 115, 121, 121, 115, 121, 121, 111, 118, 112, 113, 120, 114, 119, 123, 116, 119, 123, 116, 115, 120, 110, 116, 121, 111, 117, 125, 112, 114, 122, 109, 113, 121, 108, 112, 120, 107, 114, 122, 109, 111, 119, 106, 119, 124, 112, 121, 126, 114, 123, 125, 114, 123, 125, 114, 127, 126, 111, 122, 121, 106, 119, 118, 101, 124, 

In [87]:
pc_msg = data_params["pc_floor_msgs"][896]
points_obj = pc2.read_points(pc_msg, skip_nans=True, field_names=("x", "y", "z"))
pc = np.array(list(points_obj), dtype=np.float32)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
o3d.visualization.draw_geometries([pcd])

In [86]:
print(data_params["pc_floor_msgs"][896])

header: 
  seq: 92622
  stamp: 
    secs: 1720112669
    nsecs: 423948204
  frame_id: "odom"
height: 1
width: 584
fields: 
  - 
    name: "x"
    offset: 0
    datatype: 7
    count: 1
  - 
    name: "y"
    offset: 4
    datatype: 7
    count: 1
  - 
    name: "z"
    offset: 8
    datatype: 7
    count: 1
is_bigendian: False
point_step: 16
row_step: 9344
data: [160, 3, 151, 192, 207, 88, 187, 64, 4, 63, 210, 188, 0, 0, 128, 63, 139, 214, 150, 192, 230, 23, 187, 64, 148, 175, 188, 188, 0, 0, 128, 63, 156, 112, 22, 192, 254, 119, 149, 64, 112, 221, 219, 59, 0, 0, 128, 63, 132, 71, 36, 192, 226, 252, 152, 64, 92, 87, 246, 60, 0, 0, 128, 63, 26, 118, 47, 192, 28, 59, 156, 64, 30, 183, 95, 61, 0, 0, 128, 63, 23, 173, 150, 192, 238, 186, 186, 64, 52, 1, 244, 188, 0, 0, 128, 63, 216, 26, 151, 192, 139, 238, 186, 64, 20, 93, 170, 188, 0, 0, 128, 63, 187, 32, 43, 192, 182, 91, 154, 64, 158, 39, 39, 61, 0, 0, 128, 63, 178, 220, 51, 192, 242, 204, 156, 64, 62, 15, 126, 61, 0, 0, 128, 63, 100, 2